In [93]:
#Cleaned Up april 2 12:18 pm
#chubchub accountID UWPqS-JRVtQduqh4cFKkJnaBq_sF1RS7VtuN2I9i3tTlqSCM2jMgRXzd
#id: NxOBT_XFlCr3v_APLk_RaS_uB_nb_sR7HN024lxc2EwW4zRI
#Avg LP for each win and loss
#Past 15 games wr%
#Avg rank of players in the game
    #Get rid of self, get rid of games with friends,get solo duo rank bc it is mixed with flex
import requests
import time
import pandas as pd
APIKEY = 'RGAPI-785997ae-99c5-4778-88d5-f458ac920de5'
def getAccountID(summonerName):
    URL = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + summonerName + "?api_key=" + APIKEY
    response = requests.get(URL)
    sumId = response.json()['accountId']
    return sumId
def getSummonerID(summonerName):
    URL = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + summonerName + "?api_key=" + APIKEY
    response = requests.get(URL)
    sumId = response.json()['id']
    return sumId

def listOfRankedGames(summonerName):
    sumId = getAccountID(summonerName)
    URL = "https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/" + sumId + "?queue=420" + "&api_key=" + APIKEY
    response = requests.get(URL)
    numGames = min(15,len(response.json()['matches']))
    gameIdList = []
    for matchNum in range(numGames):
        gameIdList.append(response.json()['matches'][matchNum]["gameId"])
    return gameIdList
'''
def winningTeam(gameId):
    URL = "https://na1.api.riotgames.com/lol/match/v4/matches/" + str(gameId) + "?api_key=" + APIKEY
    response = requests.get(URL)
    #seeing if first team wins
    if response.json()["teams"][0]["win"] == "Fail":
        winner = 200
    else:
        winner = 100
    return winner
'''
def winRate(summonerName):
    URL = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + summonerName + "?api_key=" + APIKEY
    response = requests.get(URL)
    sumId = response.json()['accountId']
    #From List Of Ranked Games
    URL = "https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/" + sumId + "?queue=420" + "&api_key=" + APIKEY
    response = requests.get(URL)
    numGames = min(15,len(response.json()['matches']))
    gameIdList = []
    for matchNum in range(numGames):
        gameIdList.append(response.json()['matches'][matchNum]["gameId"])
    #Up to now gives list of 15 game IDs
    listOfWinRates = []
    for i in range(numGames):
        URL = "https://na1.api.riotgames.com/lol/match/v4/matches/" + str(gameIdList[i]) + "?api_key=" + APIKEY
        #FIGURE OUT WHICH TEAM WINS
        response = requests.get(URL)
        #seeing if first team wins
        if response.json()["gameDuration"] < 301:
            winner = 300
        elif response.json()["teams"][0]["win"] == "Fail":
            winner = 200  
        else:
            winner = 100
            
        #END, FROM WINNER METHOD ABOVE
        for j in range(0,10):
            if response.json()['participantIdentities'][j]["player"]["accountId"] == sumId:
                participantId = response.json()['participantIdentities'][j]["participantId"]
                if(participantId>5):
                    team = 200
                else:
                    team = 100
                if(winner == 300):
                    listOfWinRates.append(2)
                elif(team == winner):
                    listOfWinRates.append(1)
                else:
                    listOfWinRates.append(0)
                break
    return listOfWinRates
    


In [106]:
#KXoOd1XABkNyHVpXs_HuYLpSC3a9oieucziuBllYdlR2-if5
def tierAverage(summonerName):
    URL = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + summonerName + "?api_key=" + APIKEY
    response = requests.get(URL)
    ID = response.json()['id']
    #recent
    firstdict = {}
    listOfFriends = []
    gameIds = listOfRankedGames(summonerName)
    avg_len = len(gameIds)//3
    remainder = len(gameIds)%3
    for i in range(avg_len):
        URL = "https://na1.api.riotgames.com/lol/match/v4/matches/" + str(gameIds[i]) + "?api_key=" + APIKEY
        response = requests.get(URL)
        for j in range(0,10):
            if response.json()['participantIdentities'][j]["player"]["summonerId"] != ID:
                summonerId = response.json()['participantIdentities'][j]["player"]["summonerId"]
                URL2 = "https://na1.api.riotgames.com/lol/league/v4/entries/by-summoner/" + str(summonerId) + "?api_key=" + APIKEY  
                response2 = requests.get(URL2)
                if(len(response2.json()) == 1):
                    pts = ptsCalculator(response2.json()[0]["tier"],response2.json()[0]["rank"],response2.json()[0]["leaguePoints"])
                else:
                    if response2.json()[0]["queueType"] == "RANKED_FLEX_SR":
                        pts = ptsCalculator(response2.json()[1]["tier"],response2.json()[1]["rank"],response2.json()[1]["leaguePoints"])
                    else:
                        pts = ptsCalculator(response2.json()[0]["tier"],response2.json()[0]["rank"],response2.json()[0]["leaguePoints"])
                if(summonerId in firstdict.keys()):
                    listOfFriends.append(summonerId)
                firstdict[summonerId] = pts
    print(unique(listOfFriends))
    for i in unique(listOfFriends): #had to use unique on listOfFriends because there were multiple copies of a single friend
        del firstdict[i]            #after it deleted the friend in firstdict it kept looking to delete that same friend again
    sum = 0
    for x in firstdict:
        sum += firstdict[x]
    return (sum/len(firstdict)) #####
   
    

In [3]:
import numpy as np 
  
# function to get unique values 
def unique(list1): 
    x = np.array(list1) 
    return np.unique(x)
unique([0,0,0,0,5,5,6])

array([0, 5, 6])

In [4]:
def ptsCalculator(tier, rank, lp):
    sum = 0
    if(tier == "BRONZE"):
        sum += 4
    elif(tier == "SILVER"):
        sum += 8
    elif(tier == "GOLD"):
        sum += 12
    elif(tier == "PLATINUM"):
        sum += 16
    elif(tier == "DIAMOND"):
        sum += 20
    elif(tier == "MASTER"):
        sum += 24
    elif(tier == "GRANDMASTER"):
        sum += 28
    elif(tier == "CHALLENGER"):
        sum += 32
    if(rank == "III"):
        sum += 1
    if(rank == "II"):
        sum += 2
    if(rank == "I"):
        sum += 3
    sum += (lp/100.0)
    return sum

In [69]:
winRate("shamu9")

[1, 1, 1, 1, 0, 0, 2, 0, 1, 0, 0, 1, 0, 0, 0]

In [50]:
winRate("Unstable JG Main")
tierAverage("Unstable JG Main")

['Ztp2-vaHDEy8JupjmLpd8VLwubOkJHO-U7ZCkG26feYab5as']
657.9499999999998
40


16.448749999999997

In [51]:
tierAverage("Unstable JG Main")

['Ztp2-vaHDEy8JupjmLpd8VLwubOkJHO-U7ZCkG26feYab5as']
658.1899999999999
40


16.454749999999997

In [10]:
listOfRankedGames("Unstable JG Main")

[3334649965,
 3334575413,
 3334529101,
 3334490429,
 3334413270,
 3334194651,
 3334065974,
 3333945345,
 3333886329,
 3333850355,
 3333770890,
 3333760626,
 3333643896,
 3325384623,
 3322614823]

In [25]:
finaldata = [[0,0,0], 
             [0,0,0]]

In [27]:
finaldata[0].append(3)
finaldata

[[0, 0, 0, 3], [0, 0, 0, 3]]

In [102]:
def createList(rows, columns):
    list = [[]]
    for i in range(rows):
        for j in range(columns):
            list[i].append(0)
        if i != rows-1:
            list.append([])
    return list
finaldata = createList(1,18)

In [98]:
def fillRow(summonerName, listName,lp1,lp2,lp3,lpFinal):
    winRateList = winRate(summonerName)
    listAppended = [round(tierAverage(summonerName),3)]
    for i in winRateList:
        listAppended.append(i)
    listAppended.append(lp1)
    listAppended.append(lp2)
    listAppended.append(lp3)
    listAppended.append(lpFinal)
    listName.append(listAppended)
fillRow("RIPgear", finaldata, 20,19,19) #RIPgear 20,19,19,-16,-17
#Silver 2 82 LP / 27W 25L Win Ratio 52%
# 65 LP

[]


In [105]:
def fillRow(summonerName, listName,lpList,lpFinal):
    winRateList = winRate(summonerName)
    #listAppended = [round(tierAverage(summonerName),3)]
    for i in winRateList:
        listAppended.append(i)
    wins = []
    losses = []
    for j in lpList:
        if j > 0:
            wins.append(j)
        else:
            losses.append(j*-1)
    listAppended.append(sum(wins)/len(wins))
    listAppended.append(sum(losses)/len(losses))
    listAppended.append(lpFinal)
    listName.append(listAppended)
fillRow("RIPgear", finaldata, [20,19,-19], 19) 
#Notes - Tier Avg, 15 Win Rates, Avg Win LP, Avg Lose LP, Lp Final
#forget abt tier avg for now, come back to it later if need to
#Should be WR for last 5 games and WR for 15 Games
# TIER AVG HAS TO BE SOLO

['br4LDBRNX-kf9FUQyzrJ7cLIpUEnZt5Zxi7nzRjtzoQsSqY']


In [99]:
finaldata

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [9.033777777777779,
  1,
  1,
  1,
  1,
  0,
  0,
  2,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  30,
  28,
  27,
  26],
 [9.034, 1, 1, 1, 1, 0, 0, 2, 0, 1, 0, 0, 1, 0, 0, 0, 30, 28, 27, 26]]

In [63]:
finaldata = np.asarray(finaldata)

In [36]:
finaldata

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [18]:
df2 = pd.DataFrame(finaldata,
                   columns=['lp', 'b', 'c'])

In [19]:
df2

,lp,b,c
0,0,0,0
1,0,0,0


In [52]:
a_2d_list = [[1, 2], [3, 4]]

a_2d_list.append([5, 6])

In [53]:
a_2d_list

[[1, 2], [3, 4], [5, 6]]

In [54]:
a_2d_list.append([4])

In [55]:
a_2d_list

[[1, 2], [3, 4], [5, 6], [4]]